# Trying to improve the model in the high p_T (top) areas that it curently struggles in

-> fit an A*exp(b*p_t) to p_t bins
-> multiply wgts by exp(b*p_T)/max(exp(b*p_T)) -> low p_t -> very low wgts ; high p_T -> weights +/- 1 
        -> Problem: most weights very very small -> large loss of statistics
            -> normalizing to mean of 1 -> some small percentage of weights get very large (>2000), but most are below 1

-> IDEA: Use 10M events with these weigths and 10M with (scaled (20%)) org weights, since we don't want to lose the low p_t accuracy but also want the network to learn these high p_T events
    For NNLO use same events with new weights, NLO use different events (since we have them)



In [1]:
# import system modules
import sys
import os
os.system('for a in /sys/bus/pci/devices/*; do echo 0 | tee -a $a/numa_node>/dev/null; done') # get rid of NUMA node warnings: https://github.com/tensorflow/tensorflow/issues/42738
import gc

# import standard numerical modules
import numpy as np
import math

# import machine learning modules
import tensorflow as tf
import keras.backend as K

gpu = tf.config.list_physical_devices('GPU') # make sure GPU usage is enabled
tf.config.experimental.set_virtual_device_configuration(gpu[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=int(7.5*1024))])
print(gpu) 

sys.path.append('../') # path th DCTR.py
import DCTR


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
from scipy.optimize import curve_fit
from scipy import stats
import matplotlib.pyplot as plt

def exponential(x, A, b):
    value = A*np.exp(np.multiply(x,b))
    return value

def get_exponential_wgts(X, part_index, arg_index, cut_off = 600, div = 31):
    bins = np.linspace(min(X[:,part_index,arg_index]),cut_off, div)
    n, bins = np.histogram(X[:,part_index,arg_index], bins=bins)
    bin_centers = bins[:-1]+0.5*(bins[1:]-bins[:-1])
    max_n = max(n)
    where_max=np.squeeze(np.where(n == max_n))
    # print(bins[where_max])
    
    # fit exp to histogram
    p0 = (5e6, -0.015)
    popt, pcov = curve_fit(exponential, bin_centers[where_max:], n[where_max:], p0)

    # calculate the wgts as the inverse of the exponential and the max value as normalization
    # only use events within cut_off
    x_cut = [pt for pt in X[:,part_index,arg_index] if pt <= cut_off]
    
    max_exp = max(exponential(x_cut, 1, -1*popt[1]))
    
    # set wgts above cut off to 1
    wgts = []
    for i in range(len(x0_1)):
        if X[i,part_index,arg_index] >= cut_off:
            wgts.append(1)
        else:
            wgts.append(exponential(X[i,part_index,arg_index], 1, -1*popt[1])/max_exp)

    return wgts


# load data

In [3]:
# directory with pre converted lhe files as numpy arrays
data_dir = '../Data' # modify as needed


In [4]:
# Load POWHEG hvq x0 datasets
# x0_nrm for training, x0_plt and x0_plt_nrm for calculating stats used to decide which model performs best
# only contain tt-pair; every event has order: 
    # tt-pair, top, anti-top
# every particle has arguments: 
    # [pt, y, phi, mass, eta, E, PID, w, theta]
    # [0 , 1, 2  , 3   , 4  , 5, 6  , 7, 8    ]

# POWHEG hvq

# unnormalized dataset for checking which events fall into one (or more) of the categories that should get higher weights
x0_1 = []
x0_1 = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_converted_lhe.npz', i=3)[:9553938] # 9553938 num of NNLO samples
print(f'POWHEG hvq x0_1.shape:         {x0_1.shape}')

x0_2 = []
x0_2 = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_converted_lhe.npz', i=3)[int(1e7):int(1e7 + 9553938)] # 10M different samples than above
print(f'POWHEG hvq x0_2.shape:         {x0_2.shape}')


POWHEG hvq x0_1.shape:         (9553938, 3, 9)
POWHEG hvq x0_2.shape:         (9553938, 3, 9)


In [5]:
# same data as above, but normalized
x0_1_nrm = []
x0_1_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_normed_converted_lhe.npz', i=3)[:9553938] 
print(f'POWHEG hvq x0_nrm_1.shape:     {x0_1_nrm.shape}')

x0_2_nrm = []
x0_2_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_normed_converted_lhe.npz', i=3)[int(1e7):int(1e7 + 9553938)] # 10M different samples than above
print(f'POWHEG hvq x0_nrm_1.shape:     {x0_2_nrm.shape}')


POWHEG hvq x0_nrm_1.shape:     (9553938, 3, 9)
POWHEG hvq x0_nrm_1.shape:     (9553938, 3, 9)


In [6]:
print(gc.collect()) # gabage collection


44


In [7]:
# MiNNLO x1
# training data
x1_1_nrm = []
x1_1_nrm = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/normed_converted_lhe.npz', i=3)
print(f'MiNNLO x1_1_nrm.shape: {x1_1_nrm.shape}')

x1_2_nrm = x1_1_nrm.copy()
print(f'MiNNLO x1_2_nrm.shape: {x1_2_nrm.shape}')


MiNNLO x1_1_nrm.shape: (9553938, 3, 9)
MiNNLO x1_2_nrm.shape: (9553938, 3, 9)


In [8]:
# plotting datasets (for calculating stats during super epoch)

# POWHEG hvq
# plotting data; different from training data
x0_plt = []
x0_plt = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_converted_lhe.npz', i=3)[:9553938]
print(f'POWHEG hvq x0_plt.shape:     {x0_plt.shape}')


x0_plt_nrm = [] # nrm data for calculating rwgt in statistics phase of super epoch
x0_plt_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_normed_converted_lhe.npz', i=3)[:9553938]
print(f'POWHEG hvq x0_plt_nrm.shape: {x0_plt_nrm.shape}')


# MiNNLO
x1_plt = []
x1_plt = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/converted_lhe.npz', i=3)
print(f'MiNNLO x1_plt.shape: {x1_plt.shape}')


POWHEG hvq x0_plt.shape:     (9553938, 3, 9)
POWHEG hvq x0_plt_nrm.shape: (9553938, 3, 9)
MiNNLO x1_plt.shape: (9553938, 3, 9)


In [9]:
print(gc.collect()) # gabage collection


44


In [12]:
# get exponential weights
x0_wgts_top = get_exponential_wgts(x0_1, 1, 0) # exp wgts for top pt
x0_wgts_anti_top = get_exponential_wgts(x0_1, 2, 0) # exp wgts for anti-top pt

x0_exp_wgts = np.add(x0_wgts_top, x0_wgts_anti_top)


# X1 MiNNLO
x1_wgts_top = get_exponential_wgts(x1_plt, 1, 0) # exp wgts for top pt
x1_wgts_anti_top = get_exponential_wgts(x1_plt, 2, 0) # exp wgts for anti-top pt

x1_exp_wgts = np.add(x1_wgts_top, x1_wgts_anti_top)


# normalized event generator weights
x0_1_nrm_wgts = x0_1[:,0,-2]
x0_1_nrm_wgts /= np.mean(x0_1_nrm_wgts)

x0_2_nrm_wgts = x0_2[:,0,-2]
x0_2_nrm_wgts /= np.mean(x0_2_nrm_wgts)

x1_nrm_wgts = x1_plt[:,0,-2]
x1_nrm_wgts /= np.mean(x1_nrm_wgts)


In [13]:
# set weights for first datasets to +/- 1 +/- exp_wgt
    # -> high p_T events have |wgt| of 3, bulk events of 1
# x0_1
for i, wgt in enumerate(x0_1_nrm[:,0,-2]):
    if wgt >=0:
        wgt = 1 + x0_exp_wgts[i]
    else: 
        wgt =-1 - x0_exp_wgts[i]

 # x1_1
for i, wgt in enumerate(x1_1_nrm[:,0,-2]):
    if wgt >=0:
        wgt = 1 + x1_exp_wgts[i]
    else: 
        wgt =-1 - x0_exp_wgts[i]

# set normalized event gen wgts for second dataset
x0_2_nrm[:,0,-2] = x0_2_nrm_wgts
x1_2_nrm[:,0,-2] = x1_nrm_wgts

# create concatenated x0 and x1 datasets with normal and exp weights
x0_nrm = np.concatenate((x0_1_nrm, x0_2_nrm))
x1_nrm = np.concatenate((x1_1_nrm, x1_2_nrm))

x0_nrm[:,0,-2] /= np.mean(x0_nrm[:,0,-2]) # adjust wgts so mean is 1 across entire concated datasets incorporating above multipliers
x1_nrm[:,0,-2] /= np.mean(x1_nrm[:,0,-2])

print(f'hvq    x0_nrm.shape: {x0_nrm.shape}')
print(f'MiNNLO x1_nrm.shape: {x1_nrm.shape}')


hvq    x0_nrm.shape: (19107876, 3, 9)
MiNNLO x1_nrm.shape: (19107876, 3, 9)


In [14]:
print(gc.collect()) # gabage collection


0


In [15]:
# delete eta (pseudorapidity) and Energy -> Train only with [pt, y, phi, m, PID]

# delete energy
x0_nrm = np.delete(x0_nrm, 5, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 5, -1)
x1_nrm = np.delete(x1_nrm, 5, -1)

# delete eta
x0_nrm = np.delete(x0_nrm, 4, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 4, -1)
x1_nrm = np.delete(x1_nrm, 4, -1)


In [16]:
print(gc.collect()) # gabage collection


0


In [17]:
# prep arrays for training
x_train, x_val, y_train, y_val, wgt_train, wgt_val = DCTR.prep_arrays(x0_nrm, x1_nrm, val=0.2)

# bring into shape for training loop
train_data = (x_train, y_train, x_val, y_val, wgt_train, wgt_val)
plt_data = (x0_plt , x0_plt_nrm, x1_plt, x1_nrm_wgts)


2024-04-17 11:27:19.109651: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 11:27:19.673483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7680 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5
2024-04-17 11:27:19.684312: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3668712240 exceeds 10% of free system memory.
2024-04-17 11:27:20.797980: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 917178000 exceeds 10% of free system memory.


In [ ]:
K.clear_session() # clear gpu memory

# clear temp arrays and variables from memory
del wgt_train, wgt_val, x0_1, x0_1_nrm, x0_1_nrm_wgts, x0_2, x0_2_nrm, x0_2_nrm_wgts, x0_exp_wgts, x0_nrm
del x0_plt, x0_plt_nrm, x0_wgts_anti_top, x0_wgts_top, x1_1_nrm, x1_2_nrm, x1_exp_wgts, x1_nrm
del x1_nrm_wgts, x1_plt, x1_wgts_anti_top, x1_wgts_top, x_train, x_val, y_train, y_val


In [26]:
print(gc.collect()) # gabage collection


0


# training loop

In [25]:
train_dir = './train_20240417' # where to save models during training


In [28]:
best_model = '../best_model.tf'

# start training loop
''' train_loop() necessary arguments
train_data, plt_data

default arguments:
model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, super_patience = 5, epochs = 8, starting_super_epoch = 1, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'gelu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001

returns: best_model_list, lowest_chi2_list, lowest_loss_list
'''
best_model_list, lowest_chi2_list, _ = DCTR.train_loop(train_data, plt_data, model=best_model, batch_sizes=[8*8192, 16*8192, 24*8192], repeat=7, super_epochs=10,
                                                       train_dir = train_dir, epochs=15, learning_rate=0.001)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


starting super_epoch 1

starting training with batch_size: 65536 and 15 epochs
starting with weights from model: ../best_model.tf
starting run 0 of super_epoch 1 with batch_size 65536


2024-04-17 11:30:52.169164: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 7.50G (8053063680 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


loaded neural network model: ../best_model.tf


2024-04-17 11:31:12.972592: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets

Epoch 7: ReduceLROnPlateau reducing learning rate to 9.586202577338553e-06.
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets

Epoch 13: ReduceLROnPlateau reducing learning rate to 5.581732511927839e-06.
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
clearing keras session and collecting garbage

 best loss 0.2

In [29]:
# same thing, but with dropout of 20% (and less repeats to save time)
train_dir = './train_20240417_dropout' # where to save models during training

best_model = '../best_model.tf'

# start training loop
''' train_loop() necessary arguments
train_data, plt_data

default arguments:
model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, super_patience = 5, epochs = 8, starting_super_epoch = 1, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'gelu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001

returns: best_model_list, lowest_chi2_list, lowest_loss_list
'''
best_model_list, lowest_chi2_list, _ = DCTR.train_loop(train_data, plt_data, model=best_model, batch_sizes=[8*8192, 16*8192, 24*8192], repeat=5, super_epochs=10,
                                                       train_dir = train_dir, epochs=15, learning_rate=0.001, dropout=0.2)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


starting super_epoch 1

starting training with batch_size: 65536 and 15 epochs
starting with weights from model: ../best_model.tf
starting run 0 of super_epoch 1 with batch_size 65536
loaded neural network model: ../best_model.tf
INFO:tensorflow:Assets written to: ./train_20240417_dropout/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417_dropout/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets

Epoch 7: ReduceLROnPlateau reducing learning rate to 9.586202577338553e-06.
INFO:tensorflow:Assets written to: ./train_20240417_dropout/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417_dropout/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240417_dropout/super_epoch_1/run_0/s-1_b-65536_r-0.tf/assets

Epoch 13: ReduceLROnPlateau reducing learning rate to 5.581732511927839e-06.
INFO:tensorflow:Assets written to: ./train_20240417_dropout/super_epoch_1/run_0/s-1_b